<a href="https://colab.research.google.com/github/pedroh6s/Conhecendo-o-PANDAS/blob/main/Roteiro_projeto_b%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTAÇÃO DAS BIBLIOTECAS**

É importante realizar a instalação do pacote da biblioteca pandera

In [114]:
pip install pandera

In [115]:
import pandas as pd
import pandera as pa

Extração dos dados de uma fonte ou fontes.

É possível e importante já fazer alguns tratamentos básicos nos dados durante a conversão, como transformar os tipos de dados de cada coluna.

Nesse caso realizou-se a conversão da coluna de datas e além do tipo de dados, convertou-se também o padrão da data pra o padrão brasileiro. 

In [116]:
df = pd.read_csv("/content/drive/MyDrive/FONTES DE DADOS/ocorrencia_2010_2020 - ocorrencia_2010_2020.csv", sep=";", parse_dates=['ocorrencia_dia'], dayfirst=True)
backup = df[:]

In [117]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

O comando **pd.unique(df['nome da coluna'])** retorna um lista contendo todos os valores únicos para a coluna escolhida.

Também pode ser escrito assim **df['nome da coluna'].unique()**

Além da lista, a função também retorna o tipo de dados da coluna.

Como o output dessa função é uma lista, é possível usar métodos de listas paa tratar o output e conseguir novas informações como tamanho, ordem alfabética, etc.

É possível colocar o comando unique dentro do comando **sorted()** porém o resultado não trará mais o tipo de dados, uma vez que essa informação está fora da lista que o comando retorna.

Usando o comando sorted o resultado será apenas a lista de valores alfabeticamente.


In [118]:
pd.unique(df['ocorrencia_uf'])

array(['RJ', 'PA', 'MT', 'RS', 'BA', 'AM', 'PR', 'MG', 'SP', 'DF', 'AC',
       'TO', 'SC', 'GO', 'MA', 'ES', 'PE', 'RR', 'MS', 'CE', 'AP', 'AL',
       'RO', 'SE', 'PB', 'PI', 'RN', '**'], dtype=object)

É interessante anotar as inconsistências ou prováveis inconsistências encontradas para em seguida analisá-las.

Isso garante a integridade dos dados.


Nesse dataframe foram encontradas as seguintes inconsistências.

Coluna ocorrencia_uf: 
```
"**"
```

COluna ocorrencia_aerodromo:

```
 '****', '*****', '####', '###!'
```




In [119]:
df.loc[df.ocorrencia_uf == '**']

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
2227,49474,49474,ACIDENTE,ÁGUAS INTERNACIONAIS,**,****,2013-09-02,02:54:00,0


In [120]:
df.replace(['**', '****', '*****', '####', '###!'], pd.NA, inplace=True)

É preferível ter um dado nulo do que um dado sujo.

In [121]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  1
ocorrencia_aerodromo        2349
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

In [122]:
df

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


O método **dropna** retira todas as linhas com resultado <NA>, portanto resultados nulos.

**df.dropna()**

OBS: para executar as mudanças é necessário informar o parâmetro **inplace=True**

Isso resulta em uma perda de dados.

É necessário analisar se é melhor ter dados incompletos, ou dados a menos.

No caso desse banco de dados específico seriam perdidas mais de 2 mil linhas, o que é um percentual altíssimo que muito provavelmente alteraria consideravelmente essa análise.

In [123]:
df.dropna()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
...,...,...,...,...,...,...,...,...,...
5743,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5746,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


A função **dropna** pode ser usada para deletar subsets específicos, ou seja, podemos escolher em qual ou quais colunas procuras os valores desejados e deletar apenas as linhas que possuem aquele valor específico em uma certa coluna.

**df.dropna(subset=['nome da coluna'])**

In [124]:
#df.dropna(['ocorrencia_uf'])

TypeError: ignored

A função **drop_duplicates** vai procurar registros duplicados, ou seja, quando todos os valores da linha são iguais e caso encontre vai manter apenas a primeira ocorrência daquele registro.

**df.drop_duplicates()**

**OBS:** Para executar as mudanças é necessário o parâmetro **inplace=True**


In [128]:
df.drop_duplicates()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5747,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5748,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,<NA>,2020-12-30,18:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0
5750,79756,79756,INCIDENTE GRAVE,VICENTINA,MS,<NA>,2020-12-31,09:00:00,0


**VALIDAÇÃO DE DADOS**

É realizado de acordo com um "Schema" definido onde são colocados para cada entrada de dados um "match" em caso de incorrer uma não validação o validador acusa onde está o dado em conflito. 

Em geral junto ao pandas usamos a biblioteca Pandera com o comando 

**import pandera as pa**

Devemos informar o tipo de dados esperado para ser informado em cada coluna do seu dataframe.

O tipo object é informado como string.

Podemos verificar a quantidade de dígitos em uma coluna usando os parâmetros:

**pa.Check.str_length(tamanho_mínimo, tamanho_máximo)**

Caso existam valores nulos em uma determinada coluna é necessário que isso informado no esquema é para isso utilizados o parâmetro **nullable**

Tipo de dados **.DateTime** já faz uma validação básica:
dias até 31, mês até 13, ano não negativo. Caso a coluna esteja definido como objeto, ou seja, sem o parse de data, essa validação será rejeitada. 


**EXPRESSÕES REGULARES**

**r''** é a notação para expressões regulares



In [125]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia":pa.Column(pa.Int),
        "codigo_ocorrencia2":pa.Column(pa.Int),
        "ocorrencia_classificacao":pa.Column(pa.String),
        "ocorrencia_cidade":pa.Column(pa.String),
        "ocorrencia_uf":pa.Column(pa.String,pa.Check.str_length(2,2),nullable=True),
        "ocorrencia_aerodromo":pa.Column(pa.String,pa.Check.str_length(4,4), nullable=True),
        "ocorrencia_dia":pa.Column(pa.DateTime),
        "ocorrencia_hora":pa.Column(pa.String,pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'),nullable=True)
    }
)

Se o dataframe for exibido com o comando **schema.validate(df)** significa que o esquema foi validado. 

Caso você tenha definido os tipos de dados de maneira equivocada, você receberá um erro como por exemplo:

 **SchemaError: expected series 'codigo_ocorrencia' to have type str, got int64**


In [126]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [ ]:
#df.loc[1,'codigo_ocorrencia'] = 'oi meu chapa'
#df.loc[1,'ocorrencia_classificacao'] = 45
#df.loc[1,'ocorrencia_cidade'] = 456
#df.loc[1,'ocorrencia_uf'] = 'RDJ'
#df.loc[1,'ocorrencia_aerodromo'] = 'e lá vamos nós'
#df.loc[1,'ocorrencia_dia'] = 'teste'
#df.loc[1,'ocorrencia_hora'] = 'teste de novo'


In [ ]:
schema.validate(df)

**FILTROS**

Os filtros são apenas variáveis que estão recebendo os valores de uma procura específica em um dataframe.

A resposta dessa procura específica é uma lista contendo os valores referentes ao rótulo do index desse dataset, ou seja a coordenadas das linhas que atendem aos critérios da busca.

**filtro = df.nome_da_coluna.isnull()**

In [129]:
filtro = df.ocorrencia_aerodromo.isnull()

In [ ]:
df.loc[filtro]

Função **count** permite contar os dados validados.

O resultado de **count** e **isnull** é complementar.


In [ ]:
df.count()

**FILTROS**

Colunas que tem como tipo de dados algum tipo de número inteiro podem ser comparadas com símbolos matemáticos

```
< = > !=
```

**df.loc[filtroEscolhido, ['coluna escolhida', 'outra coluna']]**

In [ ]:
filtro1 = df.total_recomendacoes > 13
df.loc[filtro1]

In [ ]:
df.loc[filtro1, ['ocorrencia_cidade', 'ocorrencia_hora', 'total_recomendacoes']]

**AGREGAÇÃO DE FILTROS**

E para agregar dois filtros diferentes temos diferentes operadores

**&** equivale ao **e**

Quando ambas condições propostas são verdadeiras.

**|** equivale ao **ou**

Quando pelo menos uma das condições propostas é verdadeira.


In [ ]:
filtro2 = df.ocorrencia_classificacao == 'ACIDENTE'
filtro3 = df.ocorrencia_uf == 'RJ'

In [140]:
df.loc[filtro2 & filtro3]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
31,39323,39323,ACIDENTE,CASIMIRO DE ABREU,RJ,<NA>,2010-01-24,16:15:00,0
49,39295,39295,ACIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-02-06,21:09:00,0
149,40447,40447,ACIDENTE,MANGARATIBA,RJ,<NA>,2010-04-21,18:55:00,0
155,40554,40554,ACIDENTE,RESENDE,RJ,<NA>,2010-04-27,18:47:00,8
248,41729,41729,ACIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-06-25,13:05:00,0
...,...,...,...,...,...,...,...,...,...
4966,78744,78744,ACIDENTE,ITABORAÍ,RJ,<NA>,2019-06-22,12:45:00,0
5411,79309,79309,ACIDENTE,RIO DE JANEIRO,RJ,<NA>,2020-04-17,14:30:00,0
5503,79430,79430,ACIDENTE,RIO DE JANEIRO,RJ,<NA>,2020-07-06,17:40:00,0
5636,79621,79621,ACIDENTE,RIO CLARO,RJ,<NA>,2020-10-15,11:00:00,0


In [139]:
df.loc[filtro2 | filtro3]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,<NA>,2010-01-11,17:00:00,2
...,...,...,...,...,...,...,...,...,...
5740,79751,79751,ACIDENTE,PRESIDENTE CASTELO BRANCO,PR,SSXO,2020-12-27,12:30:00,0
5743,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5745,79755,79755,ACIDENTE,MATO RICO,PR,<NA>,2020-12-29,10:30:00,0
5749,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


Para informar mais de um parâmetro de busca é necessário usar a função **isin**

**df.coluna_desejada.isin(['primeiro valor', 'segundo valor])**

Busca convencional
df.coluna_desejada = 'valor procurado'

Busca multivalorada
df.coluna_desejada.isin(['primeiro valor', 'segundo valor])


In [141]:
filtro4 = df.ocorrencia_classificacao.isin(['ACIDENTE', 'INCIDENTE'])
filtro5 = df.ocorrencia_uf == 'SP'

In [ ]:
df.loc[filtro4 & filtro5]

**PROCURAS EM STRINGS**

**filtro = df.coluna_desejada.str[X] == 'Y'**

Procurando um conjunto de letras numa certa posição

Onde X é igual ao número de caracteres do valor procurado.

**filtro = df.coluna_desejada.str[-X:] == 'valor procurado'**


**PROCURAS EM STRINGS**

Em qualquer lugar 

**filtro = df.coluna_desejada.str.contains('Valor procurado')**

Procurando a letra **Y** na posição **X** 

Também podemos realizar uma pesquisa múltipla

**filtro = df.coluna_desejada.str.contains('primeiro valor|segundo valor')**

**OBS:** espaços antes ou depois do | serão contados como caracteres nas buscas.

In [ ]:
filtro10 = df.ocorrencia_cidade.str.contains('RIO|MACA')
df.loc[filtro10]